# Projeto Parcial Ciência de Dados (N1)

| Nome | RA |
| ---- | -- |
| Enzo Ferroni | 10417100 |
| Luiz Gabriel Profirio Mendes | 10382703 |



## Escolha dos Modelos

---

## Pré-processamento básico

---

## Implementação

---

## Métricas

---

## Comparação

---

## Conclusão preliminar

---